# Forbes2000 Scraper

#### Steps to get the link to the raw data:

1) Go to the website with all the companies listed - [Current Link](https://www.forbes.com/global2000/#6aeaea16335d)

2) Inspect the page, and go to the `Network` Tab

3) Refresh the page

4) Sort the scripts by size, and open the script with the raw data. It should be the largest one (around 1 mb)

5) Copy the link, __run the next cell__, and paste the link to get the scraped data in a csv

In [ ]:
import requests, json, csv, ast
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import *

link = input("What is the link for the data? ")
page = requests.get(link)
text = BeautifulSoup(page.content, 'lxml').find("p").text
data = json.loads(text)

final = []

for row in data['organizationList']['organizationsLists']:
    new = {'position': '', 'organization': '', 'country': '', 'revenue': '', 'profits': '', 'assets': '', 'marketValue': '', 'employeesList': '', }
    for k,v in row.items():
        if k in new:
            if k == 'employeesList': 
                new[k] = int(v[0])
            elif k == 'organization':
                new[k] = v['name']
            else:
                try:
                    new[k] = int(v)
                except:
                    new[k] = v
    final.append(new)
                     
output = sorted(final, key = lambda i: i['position'])

with open("forbes2000_scraped2020.csv", 'w', newline='', encoding = 'utf-8-sig') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    header = ['Rank','Company', 'Country/Territory', 'Sales ($M)', 'Profits ($M)', 'Assets ($M)', 'Market Value ($M)', 'Employees']
    spamwriter.writerow(header)
    for x in output:
        row = list(x.values())
        spamwriter.writerow(row)
        
print()
print("The file has been created!")

### Links for raw data in past years

__2020:__ https://www.forbes.com/forbesapi/org/global2000/2020/position/true.json?limit=2000

#### Flagging errors/blanks in 2020 Employees Data

In [ ]:
nums = {}

for x in output:
    if x['employeesList']: nums[x['position']] = x['employeesList']
    
nums = sorted(nums.items(), key=lambda x: x[1])[:150]
nums = [x[0] for x in nums]

for x in output:
    if x['position'] in nums:
        x['flag'] = 1
    elif not x['employeesList']:
        x['flag'] = 1
    else:
        x['flag'] = 0
        
with open("forbes2000_scraped2020.csv", 'w', newline='', encoding = 'utf-8-sig') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    header = ['Rank','Company', 'Country/Territory', 'Sales ($M)', 'Profits ($M)', 'Assets ($M)', 'Market Value ($M)', 'Employees', 'Flag']
    spamwriter.writerow(header)
    for x in output:
        row = list(x.values())
        spamwriter.writerow(row)
        
print('File has been created!')